In [ ]:
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
#now we released 2 model fine-tuning with Qlora

model_id = "airesearch/LLaMa3-8b-WangchanX-sft-Demo"
#model_id = "airesearch/typhoon-7b-WangchanX-sft-Demo"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
prompt = "ฝุ่น P.M 2.5 อันตรายอย่างไร"
messages = [
        {"role": "system", "content": "You are helpful assistant. please answer the question."},
        {"role": "user", "content": prompt }
    ]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

generated_ids = model.generate(
    input_ids=encodeds,
    max_new_tokens=500,
    early_stopping=True,
    top_k=50, top_p=0.95,
    do_sample=True,
    temperature=0.4,
    repetition_penalty = 1.2,
    eos_token_id = tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|system|>
You are helpful assistant. please answer the question.<|end_of_text|>
<|user|>
ฝุ่น P.M 2.5 อันตรายอย่างไร<|end_of_text|>
<|assistant|>
คืออนุภาคขนาดเล็กที่มีความเสี่ยงต่อสุขภาพของมนุษย์มาก โดยเฉพาะในกรณีที่มีปริมาณสูงเกินมาตรฐานทางเทคนิค มักจะพบได้ในพื้นที่ใกล้แหล่งอุตสาหกรรมหรือถ่านหินที่ใช้ก่อสร้าง ซึ่งอาจก่อให้เกิดภาวะแทรกซ้อนในการหายใจ เช่น ปอดบวม และโรคปอดเรื้อรัง รวมถึงการเพิ่มอัตราการเป็นเบาหวานและความดันโลหิตสูงทั้งในผู้ใหญ่และเด็ก นอกจากนี้ยังส่งผลกระทบร้ายแรงต่อมะเร็งเต้านม สารก่อมะเร็งน้ำลาย และมะเร็งลำไส้ที่รับรองโดยองค์การอนามัยโลก (WHO)<|end_of_text|>


In [ ]:
prompt = """Context: กรุงเทพมหานครเป็นเมืองที่มีตึกระฟ้ามากที่สุดเป็นอันดับที่ 11 ของโลกในปี พ.ศ. 2563[6] มีสถานที่ท่องเที่ยวหลากหลายในหลายรูปแบบทั้งด้านศาสนา ด้านศิลปวัฒนธรรม เช่น พระบรมมหาราชวัง ด้านการจับจ่ายซื้อของ ศูนย์การค้าต่าง ๆ หรือสถานบริการกลางคืน เช่น สยามสแควร์ ถนนข้าวสาร ที่ดึงดูดนักท่องเที่ยวต่างชาติได้อย่างมากมาย โดยในปี พ.ศ. 2562 นิตยสารฟอบส์ นิตยสารเกี่ยวกับธุรกิจและการเงินชื่อดังสัญชาติอเมริกา ได้จัดอันดับกรุงเทพมหานครเป็นเมืองที่มีนักท่องเที่ยวต่างชาติมากเป็นลำดับที่ 1 ของโลก โดยมีนักเดินทางเข้ามากว่า 22.78 ล้านคนและทำรายได้จากนักท่องเที่ยวต่างชาติมากกว่า 2,003 ล้านดอลลาร์สหรัฐ มากเป็นลำดับที่ 3 ของโลก รองจากดูไบและมักกะฮ์ตามลำดับ[7]
Question: ใครจัดอันดับกรุงเทพมหานครเป็นเมืองที่มีนักท่องเที่ยวต่างชาติมากเป็นลำดับที่ 1 ของโลก
"""
messages = [
        {"role": "system", "content": "You are helpful assistant. please answer the question."},
        {"role": "user", "content": prompt }
    ]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

generated_ids = model.generate(
    input_ids=encodeds,
    max_new_tokens=500,
    early_stopping=True,
    top_k=50, top_p=0.95,
    do_sample=True,
    temperature=0.4,
    repetition_penalty = 1.2,
    eos_token_id = tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|system|>
You are helpful assistant. please answer the question.<|end_of_text|>
<|user|>
Context: กรุงเทพมหานครเป็นเมืองที่มีตึกระฟ้ามากที่สุดเป็นอันดับที่ 11 ของโลกในปี พ.ศ. 2563[6] มีสถานที่ท่องเที่ยวหลากหลายในหลายรูปแบบทั้งด้านศาสนา ด้านศิลปวัฒนธรรม เช่น พระบรมมหาราชวัง ด้านการจับจ่ายซื้อของ ศูนย์การค้าต่าง ๆ หรือสถานบริการกลางคืน เช่น สยามสแควร์ ถนนข้าวสาร ที่ดึงดูดนักท่องเที่ยวต่างชาติได้อย่างมากมาย โดยในปี พ.ศ. 2562 นิตยสารฟอบส์ นิตยสารเกี่ยวกับธุรกิจและการเงินชื่อดังสัญชาติอเมริกา ได้จัดอันดับกรุงเทพมหานครเป็นเมืองที่มีนักท่องเที่ยวต่างชาติมากเป็นลำดับที่ 1 ของโลก โดยมีนักเดินทางเข้ามากว่า 22.78 ล้านคนและทำรายได้จากนักท่องเที่ยวต่างชาติมากกว่า 2,003 ล้านดอลลาร์สหรัฐ มากเป็นลำดับที่ 3 ของโลก รองจากดูไบและมักกะฮ์ตามลำดับ[7]
Question: ใครจัดอันดับกรุงเทพมหานครเป็นเมืองที่มีนักท่องเที่ยวต่างชาติมากเป็นลำดับที่ 1 ของโลก
<|end_of_text|>
<|assistant|>
นิตยสาร ฟอบส์ จัดอันดับกรุงเทพฯ เป็นเมืองที่มีนักท่องเที่ยวต่างชาติมากเป็นลำดับที่ 1 ของโลก ในปีพ.ศ. 2562<|end_of_text|>
